# 1) Large size vectors: lcc MAD-Mex

## a) Normalize, reproject to wgs84

In [1]:
import os

In [2]:
cur_directory = os.getcwd()

In [3]:
dir_alg_python = '/LUSTRE/MADMEX/tasks/2020/6_geonode/geonode-issue-3/python3_package_for_geonode/geonode_conabio'

In [4]:
os.chdir(dir_alg_python)

In [5]:
import fiona
from pyproj import Proj
from fiona.crs import to_string

from wrappers import reproj_normalize_and_write_large_vector


In [7]:
#direc = "/LUSTRE/MADMEX/products/landcoverchange/landsat/mosaics/2017_2018"
direc = "/LUSTRE/MADMEX/products/landcoverchange/sentinel2/2017_2018/indi50k/estados/HIDALGO/"

In [8]:
#input_filename = direc + "/madmex_landsat_changes_2017-2018.shp"
#output_filename = direc + "/madmex_landsat_changes_2017-2018_wgs84_using_fiona.shp"
#layer = "madmex_landsat_changes_2017-2018_wgs84_using_fiona"
input_filename = direc + "HIDALGO_merge.shp"
output_filename = direc + "HIDALGO_merge_wgs84_using_fiona.shp"
layer = "HIDALGO_merge_wgs84_using_fiona"
#dst_crs = "4326"
name_attribute1 = "t1_dsc_31"
name_attribute2 = "t2_dsc_31"
name_attribute3 = "t1_dsc_17"
name_attribute4 = "t2_dsc_17"
name_attribute5 = "cmb_dsc_31"
name_attribute6 = "cmb_dsc_17"

In [9]:
list_name_attributes = [name_attribute1, name_attribute2,
                        name_attribute3, name_attribute4,
                        name_attribute5, name_attribute6]

**Read and call wrapper**

In [13]:
with fiona.open(input_filename) as src:
    fc = (feature for feature in src if not None in [feature['properties'][att] for att in list_name_attributes] and
                                                     feature['geometry'] is not None)
    src_crs = to_string(src.crs)
    proj_src_crs = Proj(src_crs)
    fc_schema = src.schema
    if not proj_src_crs.crs.is_geographic:
        reproj_normalize_and_write_large_vector(fc, fc_schema,
                                                list_name_attributes,
                                                layer,
                                                output_filename,
                                                src_crs, 
                                                is_geographic = False
                                                )
    else:
        reproj_normalize_and_write_large_vector(fc, fc_schema, 
                                                list_name_attributes,
                                                layer,
                                                output_filename,
                                                src_crs
                                                )
        

## b) Register large vector layer in geonode via `shp2pgsql` and `updatelayers`

**Make sure stack deployment of geonode is up and running**

**1) Create `~/.pgpass` with contents: `hostname:port:database:username:password` and `chmod 0600 ~/.pgpass`**

In [9]:
from utils_cli import create_table_via_shp2pgsql

In [10]:
#next need to be in lowercase
name_table = "hidalgo_madmex_lcc_2017_2018_fiona" #madmex_landsat_2017-2018_lcc


In [24]:
[result_create_table, name_table, out, err] = create_table_via_shp2pgsql(output_filename, name_table)

hidalgo_madmex_lcc_2017_2018_fiona
after wait


In [25]:
print((result_create_table, out, err))

(0, None, b'')


**2) Add to geoserver from geonode_data database**. Need to follow: https://training.geonode.geo-solutions.it/004_admin_workshop/007_loading_data_into_geonode/geoserver.html

**3) Update layers using docker container `spcgeonode_django_1`**

In [11]:
from utils_docker import update_layers_via_docker

In [12]:
result_update = update_layers_via_docker(name_table)

In [14]:
#print(result_update)

**4) Make sure you are able to download it and see thumbnail. If not click to button refresh attributes and statistics for the layer in geonode. For thumbnail increase nginx conf proxy_read_timeout parameter.**

# 2) Small, medium or large size raster: lc MAD-Mex

## a) Reproject to wgs84, compress and tile it

Based on: 

https://rasterio.readthedocs.io/en/latest/topics/reproject.html

https://rasterio.readthedocs.io/en/latest/topics/image_options.html

In [5]:
from pyproj import Proj
import rasterio

from wrappers import reproj_and_write_one_band_raster

In [7]:
#direc = '/LUSTRE/MADMEX/products/landcover/sentinel2/2017/31'
#direc = '/LUSTRE/MADMEX/products/landcover/sentinel2/2017/estados/Aguascalientes/31/'
direc = '/LUSTRE/MADMEX/products/landcover/sentinel2/2017/estados/Hidalgo/31/'

In [8]:
#input_filename = ''.join([direc, '/madmex_sentinel2_2017_31.tif'])
#output_filename = ''.join([direc, '/madmex_sentinel2_2017_31_wgs84_tiled_rasterio.tif'])
#input_filename = ''.join([direc, '/Aguascalientes_2017_31.tif'])
#output_filename = ''.join([direc, '/Aguascalientes_2017_31_wgs84_tiled_rasterio.tif'])
input_filename = ''.join([direc, '/Hidalgo_2017_31.tif'])
output_filename = ''.join([direc, '/Hidalgo_2017_31_wgs84_tiled_rasterio.tif'])
#dst_crs = 'EPSG:4326'

In [10]:
with rasterio.open(input_filename) as src:
    src_crs = src.crs.to_string()
    proj_crs = Proj(src_crs)
    if not proj_crs.crs.is_geographic:
        reproj_and_write_one_band_raster(src, output_filename,
                                         is_geographic=False)
    else:
        reproj_and_write_one_band_raster(src, output_filename)

## b) Register small, medium or large raster layer in geonode via `importlayers`

**1) Importlayers cmd**

In [9]:
region = "".join(["\"Hidalgo, Mexico, North America, Latin America\""])
name = "\"sentinel2_Hidalgo_2017_31_tiled\""
title = "\"sentinel2_Hidalgo_2017_31_tiled\""
abstract = "\"Sentinel2 MAD-Mex land cover classification\""
kw = "\"MAD-Mex, Sentinel2, GeoTIFF, WCS\""


In [10]:
from utils_docker import import_layers_via_docker

In [12]:
result_import = import_layers_via_docker(region, name, title,
                                         abstract, kw,
                                         output_filename
                                         )

In [14]:
#print(result_import)

**2) Make sure you are able to download it and see thumbnail. If not click to button refresh attributes and statistics for the layer in geonode. For thumbnail increase nginx conf proxy_read_timeout parameter.**

# 3) Small to Medium size vectors: lcc MAD-Mex


## a) Normalize, reproject to wgs84

In [5]:
import geopandas as gpd
from pyproj import Proj

from wrappers import reproj_normalize_and_write_small_medium_size_vector

In [7]:
direc = '/LUSTRE/MADMEX/products/landcoverchange/sentinel2/2017_2018/indi50k/estados/AGUASCALIENTES'

In [8]:
input_file = direc + '/AGUASCALIENTES_merge.shp'
output_filename = direc + '/AGUASCALIENTES_merge_wgs84_using_geopandas.shp'
layer = 'AGUASCALIENTES_merge_wgs84_using_geopandas'

In [9]:
name_attribute1 = "t1_dsc_31"
name_attribute2 = "t2_dsc_31"
name_attribute3 = "t1_dsc_17"
name_attribute4 = "t2_dsc_17"
name_attribute5 = "cmb_dsc_31"
name_attribute6 = "cmb_dsc_17"

In [10]:
list_name_attributes = [name_attribute1, name_attribute2,
                        name_attribute3, name_attribute4,
                        name_attribute5, name_attribute6]

**Read and call wrapper**

In [13]:
gdf = gpd.read_file(input_file)
#dropNA's
gdf.dropna(subset=list_name_attributes, inplace=True)


In [14]:
#check geographic proj
crs_src = gdf.crs
src_crs = crs_src.to_string()
proj_src_crs = Proj(src_crs)
if not proj_src_crs.crs.is_geographic:
    reproj_normalize_and_write_small_medium_size_vector(gdf, 
                                                        list_name_attributes,
                                                        layer, output_filename,
                                                        is_geographic=False)
else:
    reproj_normalize_and_write_small_medium_size_vector(gdf, 
                                                        list_name_attributes,
                                                        layer, output_filename)

## b) Register vector layer in geonode via `importlayers`

**1) Importlayers cmd**

In [15]:
region = "".join(["\"Aguascalientes, Mexico, North America, Latin America\""])
name = "\"madmex_sentinel2_aguascalientes_2017_2018_lcc_2\""
title = "\"madmex_sentinel2_aguascalientes_2017_2018_lcc_2\""
abstract = "\"Sentinel2 MAD-Mex land cover change\""
kw = "\"MAD-Mex, Sentinel2, GeoTIFF, WCS\""


In [16]:
from utils_docker import import_layers_via_docker

In [17]:
result_import = import_layers_via_docker(region, name, title,
                                         abstract, kw,
                                         output_filename
                                         )

In [19]:
#print(result_import)

**2) Make sure you are able to download it and see thumbnail. If not click to button refresh attributes and statistics for the layer in geonode. For thumbnail increase nginx conf proxy_read_timeout parameter.**

# Towards python3 package

Following

https://python-packaging.readthedocs.io/en/latest/minimal.html